In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
rf_cost_model = joblib.load("../models/rf_cost_model.pkl")
xgb_co2_model = joblib.load("../models/xgb_co2_model.pkl")

In [3]:
df = pd.read_csv("../data/processed/ecopackai_feature_engineered.csv")


In [4]:
features = [
    "material_type",
    "strength",
    "weight_capacity",
    "biodegradability_score",
    "recyclability_percentage",
    "fragility_level",
    "shipping_type"
]

X = df[features]

In [5]:
X = pd.get_dummies(
    X,
    columns=["material_type", "shipping_type"],
    drop_first=True
)


X = X.reindex(columns=rf_cost_model.feature_names_in_, fill_value=0)

In [6]:
df["predicted_cost"] = rf_cost_model.predict(X)
df["predicted_co2"]  = xgb_co2_model.predict(X)


In [7]:
def min_max(series):
    return (series - series.min()) / (series.max() - series.min())

df["cost_norm"] = min_max(df["predicted_cost"])
df["co2_norm"]  = min_max(df["predicted_co2"])


In [8]:
df["ml_rank_score"] = (
    0.5 * (1 - df["cost_norm"]) +     # lower cost → higher score
    0.5 * (1 - df["co2_norm"])        # lower CO₂ → higher score
)


In [10]:
recommendations = (
    df[df["fragility_level"] == 3]
    .groupby("material_type", as_index=False)
    .agg({
        "predicted_cost": "mean",
        "predicted_co2": "mean",
        "ml_rank_score": "mean"
    })
    .sort_values("ml_rank_score", ascending=False)
    .head(5)
)

recommendations


,material_type,predicted_cost,predicted_co2,ml_rank_score
0,bagasse,0.194245,0.824416,0.609869
5,paper,0.216772,0.824629,0.548599
3,jute,0.231530,0.824460,0.508997
2,glass,0.375151,0.605152,0.502587
6,plastic,0.363300,0.624452,0.501021


In [11]:
df["ml_rank_score"] = (
    0.4 * (1 - df["cost_norm"]) +
    0.6 * (1 - df["co2_norm"])
)


In [12]:
recommendations = (
    df[df["fragility_level"] == 3]
    .groupby("material_type", as_index=False)
    .agg({
        "predicted_cost": "mean",
        "predicted_co2": "mean",
        "ml_rank_score": "mean"
    })
    .sort_values("ml_rank_score", ascending=False)
    .head(5)
)

recommendations

,material_type,predicted_cost,predicted_co2,ml_rank_score
2,glass,0.375151,0.605152,0.599699
6,plastic,0.363300,0.624452,0.585005
4,metal,0.362450,0.635662,0.563419
0,bagasse,0.194245,0.824416,0.532811
5,paper,0.216772,0.824629,0.483647
